In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("../..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/soutrik-vm-dev/code/Users/Soutrik.Chowdhury/langgraph_ventures


#### Doc Generator

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from src.utils import get_llm, get_embedder
from src.doc_loader import DocumentProcessor
import asyncio
from src.vector_db import VectorDBManager

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
llm = get_llm()
embedder = get_embedder()

In [5]:
urls = [
    "https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-march-2025",
    "https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-february-2025",
    "https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-january-2025",
    "https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-december-2024",
    "https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-november-2024",
    "https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-october-2024",
]

In [6]:
doc_processor = DocumentProcessor(llm=llm)
chunked_docs = await doc_processor.process_all(urls)
print(f"Processed {len(chunked_docs)} chunks.")

Loaded 6 documents from 6 URLs.
Split into 16 chunks.
Processed 16 chunks.


In [7]:
# Vector DB

In [8]:
vectordb_manager = VectorDBManager(
    embedder=embedder,
    drop_old=True,
    collection_name="nutmeg",
)

In [9]:
vectordb_manager.check_collections()

2025-05-15 05:34:16.376 | INFO     | src.vector_db:check_collections:120 - All collections in 'milvus_demo': ['LangChainCollection']
2025-05-15 05:34:16.377 | WARNING  | src.vector_db:check_collections:125 - Collection 'nutmeg' does not exist.


False

In [10]:
vectordb_manager.drop_collections(
    ["langchain_milvus", "book_recommendations", "lilian_weng_blog"]
)

2025-05-15 05:34:23.957 | WARNING  | src.vector_db:drop_collections:144 - Collection 'langchain_milvus' does not exist and cannot be dropped.
2025-05-15 05:34:23.957 | WARNING  | src.vector_db:drop_collections:144 - Collection 'book_recommendations' does not exist and cannot be dropped.
2025-05-15 05:34:23.958 | WARNING  | src.vector_db:drop_collections:144 - Collection 'lilian_weng_blog' does not exist and cannot be dropped.
2025-05-15 05:34:23.960 | INFO     | src.vector_db:drop_collections:149 - Remaining collections: ['LangChainCollection']


In [11]:
vectordb_manager.add_documents(chunked_docs)
vectordb_manager.check_collections()

2025-05-15 05:34:29.775 | INFO     | src.vector_db:add_documents:83 - Added 16 documents to the collection 'nutmeg'.
2025-05-15 05:34:29.779 | INFO     | src.vector_db:check_collections:120 - All collections in 'milvus_demo': ['LangChainCollection', 'nutmeg']
2025-05-15 05:34:29.780 | INFO     | src.vector_db:check_collections:122 - Collection 'nutmeg' exists.


True

In [12]:
vectordb_manager_new = VectorDBManager(
    embedder=embedder,
    collection_name="nutmeg",
    drop_old=False,
)

In [13]:
vectordb_manager_new.check_collections()

2025-05-15 05:34:37.150 | INFO     | src.vector_db:check_collections:120 - All collections in 'milvus_demo': ['LangChainCollection', 'nutmeg']
2025-05-15 05:34:37.150 | INFO     | src.vector_db:check_collections:122 - Collection 'nutmeg' exists.


True

In [14]:
vectordb_manager_new.retrieve_similar(
    query="What is the latest news about the economy?"
)

[Document(metadata={'language': 'en', 'keywords': "Nutmeg investor update October 2024 market performance central bank announcements US Federal Reserve People's Bank of China equity markets interest rates economic growth Chinese equities investment team portfolio allocation US technology stocks", 'source': 'https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-october-2024', 'title': 'Nutmeg investor update: October 2024 - Nutmeg', 'description': 'Brad Holland, Director of Investment Strategy, reviews market performance during September.', 'summary': "The Nutmeg investor update for October 2024 reviews market performance in September, highlighting significant central bank announcements from the US Federal Reserve and the People's Bank of China, which led to positive investor reactions and market movements. The update also discusses the investment team's unchanged portfolio allocation and their focus on US technology stocks due to technological change.", 'pk': '867c6724-e8fd-4692-

################# END ###################